In [15]:
# importing libraries
import pandas as pd
import numpy as np

In [17]:
# loading dataset
df = pd.read_csv('SAPS_2022_Small_Area_UR_171024.csv')

In [19]:
# Overview of the dataset
df.head()

,GUID,GEOGID,GEOGDESC,UR_Category,UR_Category_Desc,T1_1AGE0M,T1_1AGE1M,T1_1AGE2M,T1_1AGE3M,T1_1AGE4M,...,T15_1_2C,T15_1_3C,T15_1_GE4C,T15_1_NSC,T15_1_TC,T15_2_BB,T15_2_OIC,T15_2_NO,T15_2_NS,T15_2_T
0,4c07d11e-11d3-851d-e053-ca3ca8c0ca7f,017001001,017001001,4.0,4. Rural areas with high urban influence,0,6,1,1,0,...,71,20,6,7,135,80,37,9,9,135
1,4c07d11e-123a-851d-e053-ca3ca8c0ca7f,017002001,017002001,4.0,4. Rural areas with high urban influence,1,2,1,2,4,...,60,14,6,2,118,90,11,15,2,118
2,4c07d11e-14b1-851d-e053-ca3ca8c0ca7f,017002002,017002002,3.0,3. Independent urban towns,1,5,1,2,0,...,45,7,3,3,119,101,3,10,5,119
3,bf640964-28f3-4ccf-a610-04685d80ea2e,017002003/01,017002003/01,4.0,4. Rural areas with high urban influence,4,5,1,1,4,...,75,15,8,7,157,134,3,11,9,157
4,4c07d11d-f709-851d-e053-ca3ca8c0ca7f,017003001,017003001,5.0,5. Rural areas with moderate urban influence,0,1,1,1,1,...,31,19,6,9,90,50,16,14,10,90


In [23]:
# columns of interest for the analysis
columns_of_interest = [
    'UR_Category_Desc', # rural/urban description
    'T1_1AGETM', # Total male population
    'T1_1AGETF', # Total female population
    'T1_1AGETT', # Total population
    'T1_1AGE0T', # Total population age 0-14
    'T1_1AGE1T',
    'T1_1AGE2T',
    'T1_1AGE3T',
    'T1_1AGE4T', 
    'T1_1AGE5T',
    'T1_1AGE6T', 
    'T1_1AGE7T',
    'T1_1AGE8T', 
    'T1_1AGE9T', 
    'T1_1AGE10T', 
    'T1_1AGE11T', 
    'T1_1AGE12T', 
    'T1_1AGE13T',
    'T1_1AGE14T',
    'T1_1AGE65_69T', # Total population age 65+
    'T1_1AGE70_74T',
    'T1_1AGE75_79T',
    'T1_1AGE80_84T',
    'T1_1AGEGE_85T',
    'T6_1_TH' # Total no. of households
]

In [31]:
df = df[columns_of_interest]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18920 entries, 0 to 18919
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   UR_Category_Desc  18919 non-null  object
 1   T1_1AGETM         18920 non-null  int64 
 2   T1_1AGETF         18920 non-null  int64 
 3   T1_1AGETT         18920 non-null  int64 
 4   T1_1AGE0T         18920 non-null  int64 
 5   T1_1AGE1T         18920 non-null  int64 
 6   T1_1AGE2T         18920 non-null  int64 
 7   T1_1AGE3T         18920 non-null  int64 
 8   T1_1AGE4T         18920 non-null  int64 
 9   T1_1AGE5T         18920 non-null  int64 
 10  T1_1AGE6T         18920 non-null  int64 
 11  T1_1AGE7T         18920 non-null  int64 
 12  T1_1AGE8T         18920 non-null  int64 
 13  T1_1AGE9T         18920 non-null  int64 
 14  T1_1AGE10T        18920 non-null  int64 
 15  T1_1AGE11T        18920 non-null  int64 
 16  T1_1AGE12T        18920 non-null  int64 
 17  T1_1AGE13T  

In [61]:
# droppring null values, since there is only one row with null value, not having
#  a big impact on the analysis
cleaned_df = df.dropna().copy()

In [63]:
# Creating derived columns

age_0_14_cols = [f'T1_1AGE{i}T' for i in range(0,15)]
age_65_plus_cols = ['T1_1AGE65_69T', 'T1_1AGE70_74T', 'T1_1AGE75_79T', 'T1_1AGE80_84T','T1_1AGEGE_85T']

cleaned_df.loc[:, "population_0_14"] = cleaned_df[age_0_14_cols].sum(axis=1)
cleaned_df.loc[:, "population_65_plus"] = cleaned_df[age_65_plus_cols].sum(axis=1)

In [75]:
# renaming columns
cleaned_df = cleaned_df.rename(columns={
    "UR_Category_Desc": "urban_rural_category",
    "T1_1AGETM": "total_male_population",
    "T1_1AGETF": "total_female_population",
    "T1_1AGETT": "total_population",
    "T6_1_TH": "total_households"
})

In [79]:
# Creating metrics
cleaned_df["persons_per_household"] = (cleaned_df["total_population"]/cleaned_df["total_households"])
# ratio of dependents (young children and elderly)
cleaned_df["dependency_ratio"] = ((cleaned_df["population_0_14"] + cleaned_df["population_65_plus"])
                                  / cleaned_df["total_population"])

In [81]:
# dropping raw age columns
age_cols = age_0_14_cols + age_65_plus_cols
cleaned_df = cleaned_df.drop(columns=age_cols)

In [85]:
cleaned_df.to_csv("census_2022_saps_clean.csv", index=False)